In [1]:
from llmtosql.model import WikiSQLModel
from llmtosql.trainer import Trainer
from llmtosql.dataloader import WikiSQLDataset
from llmtosql.utils.utils import plot_history, load_model, load_history
from matplotlib import pyplot as plt
import sys
import os
import json
import numpy as np
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.insert(0, '..')

In [2]:
model = WikiSQLModel(base_model_type='bert-base-cased', attention_type='cross', col_drop=False)
# try:
#     model = load_model(model, 'model_output/model.pth')
# except Exception as e:
#     print(f'No model loaded - {e}')

2023-03-27 17:07:28 [info     ] Using cross attention mechanism


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
train_set =  WikiSQLDataset(type='dev', model=model)

2023-03-27 17:07:31 [info     ] Tokenizing dataset.


100%|██████████| 8421/8421 [00:08<00:00, 1026.34it/s]


In [4]:
train_set.maxcondsLength

4

In [5]:
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

loss = CrossEntropyLoss()
loader = DataLoader(train_set, batch_size=3)

In [6]:
for data in loader:
    label = data['labels']
    break

In [56]:
torch.transpose(torch.stack(label['conds'][2]), 0, 1)

tensor([[   0, -100, -100, -100],
        [   0, -100, -100, -100],
        [   0, -100, -100, -100]])

In [103]:
t = torch.transpose(torch.transpose(torch.stack([lab['input_ids'] for lab in label['conds'][3]]), 0, 1), 1, 2)

In [40]:
t.shape

torch.Size([3, 512, 4])

In [135]:
import torch

outputs = torch.randn(3, 512, 4, 50231)

In [65]:
l = .0
l += loss(torch.transpose(outputs[:,:,0,:], 1, 2), label['conds'][3][0]['input_ids']).item()

In [66]:
l

11.333290100097656

In [116]:
outputs = torch.randn(3, 512, 4)

In [62]:
l += loss(outputs, torch.transpose(torch.stack(label['conds'][1]), 0, 1)).item()

In [136]:
f = torch.nn.Softmax(dim=-1)

In [137]:
pred = torch.argmax(f(outputs), -1)

In [140]:
pred.shape

torch.Size([3, 512, 4])

In [90]:
from sklearn.metrics import accuracy_score

In [143]:
accuracy = (pred.cpu().detach().numpy() == t.cpu().detach().numpy()).all(axis=(0, 1)).mean()

In [144]:
accuracy

0.0

In [145]:
my_l = [2, 3, 4, 5]
np.average(my_l)

3.5

In [131]:
real = torch.tensor([
    [261, 232, 290, 250],
    [324, 93, 222, 0],
    [11, 473, 1, 3]
])

In [134]:
(pred.cpu().detach().numpy() == real.cpu().detach().numpy()).all(axis=(0)).mean()

0.5

In [124]:
accuracy_score(torch.transpose(torch.stack(label['conds'][1]), 0, 1).cpu().detach().numpy(), pred.cpu().detach().numpy())

ValueError: multiclass-multioutput is not supported

In [115]:
accuracy_score(t.cpu().detach().numpy(), pred.cpu().detach().numpy())

ValueError: unknown is not supported

In [105]:
t.shape

torch.Size([3, 512, 4])

In [120]:
pred.shape

torch.Size([3, 4])

In [76]:
a = torch.randn(3, 4, 6)

In [78]:
f(a)

tensor([[[0.2408, 0.0675, 0.1556, 0.1709, 0.3063, 0.0591],
         [0.1089, 0.1185, 0.0442, 0.3190, 0.1418, 0.2676],
         [0.2543, 0.2394, 0.1149, 0.0337, 0.2232, 0.1345],
         [0.2471, 0.0381, 0.2747, 0.1068, 0.0720, 0.2614]],

        [[0.0766, 0.5757, 0.0393, 0.0278, 0.0559, 0.2246],
         [0.2719, 0.0996, 0.0536, 0.1610, 0.1534, 0.2604],
         [0.1984, 0.0649, 0.2455, 0.3223, 0.0188, 0.1501],
         [0.3373, 0.1191, 0.0524, 0.2464, 0.1573, 0.0875]],

        [[0.4149, 0.0470, 0.0787, 0.1765, 0.1504, 0.1325],
         [0.0700, 0.1283, 0.4201, 0.0272, 0.0692, 0.2853],
         [0.6156, 0.0613, 0.0535, 0.0320, 0.1318, 0.1058],
         [0.0349, 0.1950, 0.5787, 0.0406, 0.0862, 0.0646]]])

In [49]:
isinstance(outputs, torch.Tensor)

True

In [5]:
# val_set = WikiSQLDataset(type='dev')

In [6]:
len(train_set.data)

8421

In [ ]:
# datasets = (train_set, val_set)

In [ ]:
config = {
    'seed': 32,
    'scheduler': None,
    'optimizer': 'adam',
    'momentum': 0.9,
    'weight_decay': 0.0,
    'lr': 0.0005,
    'criterion': 'custom',
    'metric': 'accuracy',
    'pred_function': 'softmax',
    'model_dir': 'model_output',
    'backend': 'smddp'
}

In [ ]:
trainer = Trainer(model, datasets=train_set,
                  epochs=10, batch_size=32,
                  is_parallel=False,
                  **config)

In [ ]:
trainer.fit()

In [ ]:
trainer.save_history_('model_output')

In [ ]:
history = load_history('model_output')

In [ ]:
plot_history(history)

In [ ]:
model = WikiSQLModel(base_model_type='bert-base-uncased', attention_type='sqlnet', col_drop=True)
model = load_model(model, 'model_output/model.pth')
test_loss, test_accuracy = trainer.test(model, trainer.val_loader)

In [ ]:
print(f'Model accuracy on test: {test_accuracy}')